In [4]:
import os
from urllib.request import Request, urlopen
from urllib.error import URLError
import requests, wget, re
from time import sleep
import configuration as config
from pathlib import Path

from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem

# you can also import SoftwareEngine, HardwareType, SoftwareType, Popularity from random_user_agent.params
# you can also set number of user agents required by providing `limit` as parameter

software_names = [SoftwareName.CHROME.value]
operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]   

user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)

# Get list of user agents.
user_agents = user_agent_rotator.get_user_agents()

# Get Random User Agent String.
user_agent = user_agent_rotator.get_random_user_agent()
#print(user_agent)

In [7]:
from os.path import join

webpage_dict = {}
url_list = []

download=False
#get search urls / kbb only? 
with open(join(config.INPUT_DIR, 'url_list.txt'), 'r') as f:
    search_url_list = f.readlines()

#find urls in search pages
for search_url in search_url_list:
    base_url = '/'.join(search_url.split('/')[0:3])
    if 'kbb' in base_url: 
        txt = requests.get(search_url).text
        found_urls = re.findall(r'"url":"(.*?)"', txt)
        found_urls = [x for x in found_urls if 'listingId' in x]
        
        if len(found_urls) == 0:
            with open('debug.txt','w') as f:
                print("Something's wrong")
                f.write(f"{txt} \n")
        print(f"KBB total: {len(found_urls)}")
        url_list.extend(found_urls)


    elif 'edmunds' in base_url:
        headers = {
                    'User-Agent': 'My User Agent 1.0',
                    'From': 'youremail@domain.com'  # This is another valid field
                }

        txt = requests.get(search_url, headers=headers).text
        found_urls = re.findall(r'"url":"(.*?)"', txt)

        found_urls = [x for x in found_urls if 'vin' in x]
        print(f"Edmunds total: {len(found_urls)}")
        url_list.extend(found_urls)

print(f"Total: {len(url_list)}")
seen = set()
uniq = []
for x in url_list:
    if x not in seen:
        uniq.append(x)
        seen.add(x)

url_list = uniq
print(f"Grand total after duplicates removed: {len(url_list)}")

with open('urls_out.txt', 'w') as f:
        for url in url_list:
            f.write(f"{url} \n")

i=0 
for url in url_list:
    base_url = '/'.join(url.split('/')[0:3])
    user_agent = 'My User Agent 1.0' if 'edmunds' in base_url else user_agent_rotator.get_random_user_agent()
    print(f"{i}", end=" ")
    i+=1
    req = Request(url, headers={'User-Agent':user_agent,
                                'accept':'text/html;q=0.8,application/signed-exchange;v=b3;q=0.9',
                                'accept-language':'en-US,en;q=0.9'})
    try:
        response = urlopen(req)
    except URLError as e:
        print(url)
        if hasattr(e, 'reason'):
            print('We failed to reach a server.')
            print('Reason: ', e.reason)
        elif hasattr(e, 'code'):
            print('The server couldn\'t fulfill the request')
            print('Error code:', e.code)
    else:
        try:
            if response.headers.get_content_charset() == 'utf-8':
                webpage_as_string = response.read().decode(response.headers.get_content_charset(), "ignore")
            else:
                webpage_as_string = response.read().decode(response.headers.get_content_charset(), "ignore")
        except UnicodeEncodeError as e:
            print(e)
            webpage_as_string = response.read()
        #print(webpage_as_string)
        #print(response.headers.get_content_charset())
                
        webpage_dict[url] = (webpage_as_string)


Edmunds total: 16
2
Something's wrong
KBB total: 0
208
KBB total: 206
Total: 222
Grand total after duplicates removed: 108
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 https://www.kbb.com/cars-for-sale/vehicledetails.xhtml?listingId=589249917
We failed to reach a server.
Reason:  Forbidden
38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 https://www.kbb.com/cars-for-sale/vehicledetails.xhtml?listingId=586402136
We failed to reach a server.
Reason:  Forbidden
92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 

In [6]:
import requests
from ast import literal_eval
import pandas as pd
from os.path import join

output_filename = join(config.OUTPUT_DIR, f"potential vehicles.csv")
vehicle_df = pd.DataFrame(columns=["Price", "Mileage", "Year", "Make", "Model", "Trim", 
                            "Color", "Fuel Economy", "VIN", "Location", "URL"])

if os.path.exists(output_filename):
    vehicle_df = pd.read_csv(output_filename)

for url, txt in webpage_dict.items():
    #print(url)
    base_url = '/'.join(url.split('/')[0:3])

        #kbb data scraper
    if 'kbb' in base_url:  
            #find vehicle data
        vehicle_dict = {}
        url = url.split("&")[0]
        try:
            location_dict = literal_eval(re.findall(r'"location":({.*?})', txt)[1]+"}")['address']
            #print(location_dict)
            vehicle_data_dict = literal_eval(re.findall(r'"vehicle":({.*?}),', txt)[0].replace("null",'"N/A"'))
            #print(vehicle_data_dict)
            pass
        except Exception as e:
            with open("debug.txt", "wb") as f: f.write(txt.encode('utf-8'))
            print(re.findall(r'"location":({.*?})', txt)[1]+"}")
            print(re.findall(r'"vehicle":({.*?}),', txt)[0].replace("null",'"N/A"'))
            raise Exception
            continue

            #put vehicle data into dict
        try:
            address = location_dict['address1']
        except:
            address = "N/A"
        city = location_dict['city']
        state = location_dict['state']
        zip_code = location_dict['zip']

        full_address = f"{address} {city}, {state} {zip_code}"
            
        vehicle_dict["Price"] = vehicle_data_dict['price']
        vehicle_dict["Mileage"] = int("".join(vehicle_data_dict['odometer'].split(',')).split(" ")[0])
        vehicle_dict["Year"] = vehicle_data_dict['car_year']
        vehicle_dict["Make"] = vehicle_data_dict['makeName'][0]
        vehicle_dict["Model"] = vehicle_data_dict['modelName'][0]
        vehicle_dict["Color"] = vehicle_data_dict['color'][0]
        vehicle_dict["Fuel Economy"] = " | ".join(vehicle_data_dict['fuelEconomy']).replace("mpg ", "")
        vehicle_dict["VIN"] = vehicle_data_dict['vin']
        vehicle_dict["Location"] = full_address
        vehicle_dict["URL"] = url
        
        try:
            vehicle_dict["Trim"] = vehicle_data_dict['trim']
        except:
            pass

        vehicle_df = vehicle_df.append(vehicle_dict, ignore_index=True)

    elif 'edmunds' in base_url:
            #find vehicle data
        vehicle_dict = {}
        price = re.findall(r'"vdp-price-row">(.*?)<', txt)[0][1:]
        miles = int(re.findall(r'"col">(.*?) ', txt)[0].replace(",", ""))
        year_make_model = re.findall(r'<h1 class="not-opaque text-black d-inline-block mb-0 size-24">(.*?)</h1>', txt)[0].split(" ")
        year = year_make_model[0]
        make = year_make_model[1]
        model = year_make_model[2]
        fuel_economy = re.findall(r'"MPG"></i></div><div class="col">(.*?)<', txt)[0]
        vin = url.split('/')[7]
        location_dict = literal_eval(re.findall(r'"location":({.*?})', txt)[2].replace('false', "False"))
        trim = "N/A"
        try:
            color = re.findall(r'<span>Ext: (.*?)<', txt)[0]
        except Exception as e:
            print(re.findall(r'<span>Ext: (.*?)<', txt))
            color = "N/A"
            print(e)
            #raise Exception
            continue

            #put vehicle data into dict
        latitude = location_dict['latitude']
        longitude = location_dict['longitude']
        
        full_address = f"{latitude} {longitude}"
            
        vehicle_dict["Price"] = price
        vehicle_dict["Mileage"] = miles
        vehicle_dict["Year"] = year
        vehicle_dict["Make"] = make
        vehicle_dict["Model"] = model
        vehicle_dict["Trim"] = trim
        vehicle_dict["Color"] = color
        vehicle_dict["Fuel Economy"] = fuel_economy.replace(" / ", " | ")
        vehicle_dict["VIN"] = vin
        vehicle_dict["Location"] = full_address
        vehicle_dict["URL"] = url

        vehicle_df = vehicle_df.append(vehicle_dict, ignore_index=True)

    elif '' in base_url:
        pass
        raise Exception("Stop here...")

    #add vehicle_dict to dataframe and save in csv
print(f"{len(vehicle_df)} cars found.")
vehicle_df = vehicle_df.drop_duplicates()
print(f"{len(vehicle_df)} after dupes removed")
vehicle_df.to_csv(join(output_filename), index=False)
print(f"Saved found cars to: {output_filename}")

[]
list index out of range
104 cars found.
104 after dupes removed
Saved found cars to: output\potential vehicles.csv


In [22]:
import webbrowser
for index, row in vehicle_df.iterrows():
    print(f"Row {index}: {row['Year']} {row['Make']} {row['Model']} for {row['Price']} with {row['Mileage']} miles")
    url = row["URL"]
    webbrowser.open(url, new=2)
    rating = input("Input rating (1-10): ")
    vehicle_df.at[index, 'My Rating'] = rating
output_filename = output_filename = join(config.OUTPUT_DIR, f"potential vehicles rated.csv")
vehicle_df.to_csv(join(output_filename), index=False)

Row 0: 2013 Toyota Corolla for 7,999 with 118189 miles
Row 1: 2015 Toyota Corolla for 8,995 with 152929 miles
Row 2: 2013 Toyota Corolla for 8,995 with 86647 miles
Row 3: 2010 Toyota Corolla for 7,000 with 153322 miles
Row 4: 2010 Toyota Corolla for 6,854 with 156170 miles
